<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instal and Setup (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-93uh2g5z
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-93uh2g5z
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 5e8ae5a3d533f48e2d30b6dea15bbbc9dbf7b343
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.3 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=42392 sha256=f7e05a913f41d15908eaccee173c2863385b71da0037111a93adde69f9d441df
  Stored in directory: /tmp/pip-ephem-wheel-cache-g0ev2wi2/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

In [2]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='ibd'
)

In [5]:
# @title DataFrame Operations
import yfinance as yf
import vistock.tw as tw

def yf_get_current_price(ticker):
    stock = yf.Ticker(ticker)
    historical_data = stock.history(period="1d")
    return historical_data['Close'].iloc[-1]

def is_taiwan_stock_df(df):
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    return ticker.isdigit()

def add_price_column(df):
    column_names = df.columns.tolist()
    if 'Price' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    df['Price'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Price')
    df = df[column_names]
    if is_taiwan_stock_df(df):
        df['Price'] = df['Ticker'].apply(tw.stock_price)
    else:
        df['Price'] = df['Ticker'].apply(yf_get_current_price)
    return df

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    if not is_taiwan_stock_df(df):
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df

def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.


### Execute Actions Step by Step


In [9]:
# @title Step 1. Pick a File

import ipywidgets as widgets
from IPython.display import display

# List of filenames
filenames = github.list_filenames()

# Extract and sort all unique values
dates = sorted(set(fn.split('_')[3].replace('.csv', '') for fn in filenames), reverse=True)
sources = sorted(set(fn.split('_')[0] for fn in filenames), reverse=True)
types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames for setting dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px per character for width

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create the file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

# Function to update file options based on selected filters
def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            f"{source_dropdown.value}_" in fn and
            type_dropdown.value in fn and
            period_dropdown.value in fn)
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match

# Function to handle file selection and print the selected filename
#def on_file_selected(change):
#    selected_file = change['new']
#    print(f"Selected file: {selected_file}")

# Update file options when any of the dropdowns change
date_dropdown.observe(update_file_options, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Print the selected file when it changes
#file_dropdown.observe(on_file_selected, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize the file dropdown with options
update_file_options()


Dropdown(description='Date:', layout=Layout(width='430px'), options=('20240813', '20240812', '20240811', '2024…

Dropdown(description='Source:', layout=Layout(width='430px'), options=('TWSE+TPEX', 'SPX+DJIA+NDX+SOX', 'SPX',…

Dropdown(description='Type:', layout=Layout(width='430px'), options=('stocks', 'industries'), value='stocks')

Dropdown(description='Period:', layout=Layout(width='430px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='430px'), options=(), value=None)

In [7]:
# @title Step 2. Print Top Percentile Items {"run":"auto"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
df_top = df.head(num_items).copy()
df_top = add_price_column(df_top)
df_top = add_name_column(df_top)
display(df_top)

if 'Name' in df_top.columns:
    print(list(df_top['Name']))
if 'Ticker' in df_top.columns:
    print(list(df_top['Ticker']))

Source: TWSE+TPEX
Type: stocks
Period: 2y
Date: 20240813


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,6144,得利影,,Communication Services,Entertainment,466.81,273.65,119.24,84.33,99.0,99.0,88,19,1
1,1799,易威,,Healthcare,Drug Manufacturers—Specialty & Generic,404.50,386.05,220.02,187.96,99.0,99.0,99,97,2
2,8374,羅昇,122.5,Industrials,Industrial Distribution,384.72,266.61,85.74,83.32,99.0,99.0,45,16,3
3,2365,昆盈,62.9,Technology,Computer Hardware,362.34,300.28,124.28,186.47,99.0,99.0,90,97,4
4,4562,穎漢,70.5,Industrials,Specialty Industrial Machinery,343.39,296.45,148.13,86.08,99.0,99.0,95,25,5
5,6640,均華,,Technology,Semiconductors,317.98,291.08,240.56,196.52,99.0,99.0,99,98,6
6,6442,光聖,376.5,Technology,Electronic Components,313.87,435.31,193.38,185.17,99.0,99.0,98,97,7
7,2543,皇昌,62.3,Industrials,Engineering & Construction,313.30,293.91,288.16,260.34,99.0,99.0,99,99,8
8,2348,海悅,270.0,Real Estate,Real Estate Services,280.27,227.81,203.62,121.87,99.0,98.0,99,85,9
9,4909,新復興,,Technology,Electronic Components,277.97,265.26,237.11,212.44,99.0,99.0,99,98,10


['得利影', '易威', '羅昇', '昆盈', '穎漢', '均華', '光聖', '皇昌', '海悅', '新復興']
['6144', '1799', '8374', '2365', '4562', '6640', '6442', '2543', '2348', '4909']


In [8]:
# @title Step 3. Filter Items with Increasing RS > 100
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
filtered_df = filter_increasing_relative_strength(df)
df_top = filtered_df.head(num_items).copy()
df_top = add_price_column(df_top)
df_top = add_name_column(df_top)
display(df_top)

if 'Name' in df_top.columns:
    print(list(df_top['Name']))
if 'Ticker' in df_top.columns:
    print(list(df_top['Ticker']))

Source: TWSE+TPEX
Type: stocks
Period: 2y
Date: 20240813


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,6144,得利影,,Communication Services,Entertainment,466.81,273.65,119.24,84.33,99.0,99.0,88,19,1
1,1799,易威,,Healthcare,Drug Manufacturers—Specialty & Generic,404.50,386.05,220.02,187.96,99.0,99.0,99,97,2
2,8374,羅昇,122.5,Industrials,Industrial Distribution,384.72,266.61,85.74,83.32,99.0,99.0,45,16,3
4,4562,穎漢,70.5,Industrials,Specialty Industrial Machinery,343.39,296.45,148.13,86.08,99.0,99.0,95,25,5
5,6640,均華,,Technology,Semiconductors,317.98,291.08,240.56,196.52,99.0,99.0,99,98,6
7,2543,皇昌,62.3,Industrials,Engineering & Construction,313.30,293.91,288.16,260.34,99.0,99.0,99,99,8
8,2348,海悅,270.0,Real Estate,Real Estate Services,280.27,227.81,203.62,121.87,99.0,98.0,99,85,9
9,4909,新復興,,Technology,Electronic Components,277.97,265.26,237.11,212.44,99.0,99.0,99,98,10
12,9906,欣巴巴,207.5,Industrials,Engineering & Construction,274.09,182.54,119.42,110.45,99.0,97.0,88,78,13
16,2524,京城,106.0,Real Estate,Real Estate—Development,245.69,154.52,133.02,87.93,99.0,96.0,93,32,17


['得利影', '易威', '羅昇', '穎漢', '均華', '皇昌', '海悅', '新復興', '欣巴巴', '京城']
['6144', '1799', '8374', '4562', '6640', '2543', '2348', '4909', '9906', '2524']
